In [1]:
import pandas as pd
import re

# Charger le fichier CSV
df = pd.read_csv('filtered_tweets_engie.csv', delimiter=';')

# Convertir la colonne 'created_at' en datetime
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce', utc=True)


# Supprimer la colonne 'name' (expliqué dans la consigne)
df = df.drop(columns=['name'])

# Supprimer les tweets où 'screen_name' est 'ENGIEpartFR'
df = df[df['screen_name'] != 'ENGIEpartFR']

# Supprimer les caractères spéciaux, les emojis et les sauts de ligne
def clean_text(text):
    # Enlever les sauts de ligne et les retours chariots
    text = text.replace('\n', ' ').replace('\r', ' ')
    # Enlever les caractères spéciaux
    text = re.sub(r'(https?://[^\s]+)','',text)
    text = re.sub(r'[^\w\s,!?;.\'\"àâäéèêëîïôöùûüç]', '', text)  # Garde les lettres, chiffres, accents et certains signes de ponctuation
    return text

df['full_text'] = df['full_text'].apply(clean_text)

# Supprimer les messages successifs du même 'screen_name'
df['prev_screen_name'] = df['screen_name'].shift(1)  # Créer une colonne pour le 'screen_name' précédent
df['is_duplicate'] = df['screen_name'] == df['prev_screen_name']  # Marquer les messages successifs

# Filtrer les doublons successifs
df_filtered = df[~df['is_duplicate']]  # Garder uniquement les tweets qui ne sont pas des doublons successifs

# Supprimer la colonne temporaire
df_filtered.drop(columns=['prev_screen_name', 'is_duplicate'], inplace=True)

# Renommer les 'id' pour commencer à 1 et incrémenter de 1 à chaque ligne
df_filtered['id'] = range(1, len(df_filtered) + 1)

# Sauvegarder le DataFrame nettoyé dans un nouveau fichier CSV
df_filtered.to_csv('filtered_tweets_engie_cleaned.csv', index=False, sep=';')

# Afficher les premières lignes du DataFrame nettoyé
print(df_filtered.head())

   id      screen_name                created_at  \
0   1       gptournier 2023-11-16 15:13:18+00:00   
1   2       jouanetwan 2023-11-26 07:34:34+00:00   
2   3  vince_thouvenin 2023-12-21 14:27:08+00:00   
3   4   BiduleAnatheme 2023-12-28 21:32:58+00:00   
4   5  vince_thouvenin 2023-12-29 10:08:10+00:00   

                                           full_text  
0  ENGIEpartFR n6 mois dattente et tjs aucune rép...  
1  Bonjour ENGIEpartSAV , lappli monpilotageelec ...  
2  ENGIEpartFR mon syndic de copropriété sergic e...  
3  ENGIEpartSAV vous envisagez de vous occuper de...  
4  ENGIEpartSAV retour de votre technicien "vous ...  


C:\Users\liena\AppData\Local\Temp\ipykernel_19016\2014227523.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.drop(columns=['prev_screen_name', 'is_duplicate'], inplace=True)
C:\Users\liena\AppData\Local\Temp\ipykernel_19016\2014227523.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['id'] = range(1, len(df_filtered) + 1)


In [13]:
# Ajouter les colonnes "heure", "date", "semaine", "mois", "annee" pour analyser les tweets par heure, par jour, par semaine, par mois, par année
df_filtered["date"] = df_filtered["created_at"].dt.date
df_filtered["heure"] = df_filtered["created_at"].dt.hour
df_filtered["semaine"] = df_filtered["created_at"].dt.to_period("W").astype(str)
df_filtered["mois"] = df_filtered["created_at"].dt.to_period("M").astype(str)
df_filtered["annee"] = df_filtered["created_at"].dt.to_period("Y").astype(str)

tweets_par_jour = df_filtered["date"].value_counts().sort_index()
tweets_par_semaine = df_filtered["semaine"].value_counts().sort_index()
tweets_par_mois = df_filtered["mois"].value_counts().sort_index()
tweets_par_annee = df_filtered["annee"].value_counts().sort_index()
tweets_par_heure = df_filtered["heure"].value_counts().sort_index()

print("\nNombre de tweets par heure :\n", tweets_par_heure.to_frame("Tweets"))
print("\nNombre de tweets par jour :\n", tweets_par_jour.to_frame("Tweets"))
print("\nNombre de tweets par semaine :\n", tweets_par_semaine.to_frame("Tweets"))
print("\nNombre de tweets par mois :\n", tweets_par_mois.to_frame("Tweets"))
print("\nNombre de tweets par année :\n", tweets_par_annee.to_frame("Tweets"))

# Comptage des mentions des comptes Engie
engie_accounts = ["ENGIEgroup", "ENGIEpartFR", "ENGIEpartSAV"]
mentions = {account: df_filtered["full_text"].str.contains(account, case=False, na=False).sum() for account in engie_accounts}

# Afficher les résultats des mentions
print("\nNombre de tweets mentionnant les comptes Engie :")
for account, count in mentions.items():
    print(f"{account}: {count}")

df_filtered.to_csv('filtered_tweets_engie_cleaned.csv', index=False, sep=';')

df_filtered


Nombre de tweets par heure :
        Tweets
heure        
0           4
1           2
2           1
3           3
4           1
5          10
6          17
7          44
8          41
9          36
10         46
11         36
12         40
13         25
14         37
15         34
16         28
17         29
18         21
19         36
20         13
21         17
22          5
23          3

Nombre de tweets par jour :
             Tweets
date              
2023-11-16       1
2023-11-26       1
2023-12-21       1
2023-12-28       1
2023-12-29       3
...            ...
2025-02-24       4
2025-02-25       3
2025-02-26       3
2025-03-03       5
2025-03-04       1

[286 rows x 1 columns]

Nombre de tweets par semaine :
                        Tweets
semaine                      
2023-11-13/2023-11-19       1
2023-11-20/2023-11-26       1
2023-12-18/2023-12-24       1
2023-12-25/2023-12-31       5
2024-01-01/2024-01-07      12
...                       ...
2025-02-03/2025-02-09       3
2

C:\Users\ludov\AppData\Local\Temp\ipykernel_25960\2694576204.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["date"] = df_filtered["created_at"].dt.date
C:\Users\ludov\AppData\Local\Temp\ipykernel_25960\2694576204.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["heure"] = df_filtered["created_at"].dt.hour
C:\Users\ludov\AppData\Local\Temp\ipykernel_25960\2694576204.py:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_filtered[

,id,screen_name,created_at,full_text,heure,date,semaine,mois,annee
0,1,gptournier,2023-11-16 15:13:18+00:00,ENGIEpartFR n6 mois dattente et tjs aucune rép...,15,2023-11-16,2023-11-13/2023-11-19,2023-11,2023
1,2,jouanetwan,2023-11-26 07:34:34+00:00,"Bonjour ENGIEpartSAV , lappli monpilotageelec ...",7,2023-11-26,2023-11-20/2023-11-26,2023-11,2023
2,3,vince_thouvenin,2023-12-21 14:27:08+00:00,ENGIEpartFR mon syndic de copropriété sergic e...,14,2023-12-21,2023-12-18/2023-12-24,2023-12,2023
3,4,BiduleAnatheme,2023-12-28 21:32:58+00:00,ENGIEpartSAV vous envisagez de vous occuper de...,21,2023-12-28,2023-12-25/2023-12-31,2023-12,2023
4,5,vince_thouvenin,2023-12-29 10:08:10+00:00,"ENGIEpartSAV retour de votre technicien ""vous ...",10,2023-12-29,2023-12-25/2023-12-31,2023-12,2023
...,...,...,...,...,...,...,...,...,...
571,525,djofthp,2025-03-03 15:24:10+00:00,ENGIEpartFR on vous parle,15,2025-03-03,2025-03-03/2025-03-09,2025-03,2025
572,526,julien_ducerf,2025-03-03 17:50:42+00:00,ENGIEpartFR c'est du harcèlement pour augmente...,17,2025-03-03,2025-03-03/2025-03-09,2025-03,2025
573,527,SouareFirst,2025-03-03 18:47:06+00:00,ENGIEpartFR bonjour engie les voleurs ! Attent...,18,2025-03-03,2025-03-03/2025-03-09,2025-03,2025
574,528,Leoletonneau,2025-03-03 23:25:45+00:00,"ENGIEpartFR vraiment des escrocs, des montants...",23,2025-03-03,2025-03-03/2025-03-09,2025-03,2025


In [ ]:
# Charger le fichier CSV
cdf = pd.read_csv('filtered_tweets_engie_cleaned.csv', delimiter=';')

def analyse_tweet(tweet_text):
    """
    Install an additional SDK for JSON schema support Google AI Python SDK

    $ pip install google.ai.generativelanguage
    $ pip install google.generativeai
    """

    import google.generativeai as genai
    from google.ai.generativelanguage_v1beta.types import content
    import json

    genai.configure(api_key='KEY')

    # Create the model
    generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_schema": content.Schema(
        type = content.Type.OBJECT,
        required = ["Sentiment", "Problematique", "Score", "Urgence", "Reponse_automatique", "Reparabilite", "Solution", "Type"],
        properties = {
        "Sentiment": content.Schema(
            type = content.Type.STRING,
        ),
        "Problematique": content.Schema(
            type = content.Type.STRING,
        ),
        "Score": content.Schema(
            type = content.Type.INTEGER,
        ),
        "Urgence": content.Schema(
            type = content.Type.INTEGER,
        ),
        "Reponse_automatique": content.Schema(
            type = content.Type.STRING,
        ),
        "Lieu": content.Schema(
            type = content.Type.STRING,
        ),
        "Reparabilite": content.Schema(
            type = content.Type.STRING,
        ),
        "Solution": content.Schema(
            type = content.Type.STRING,
        ),
        "Type": content.Schema(
            type = content.Type.STRING,
        ),
        },
    ),
    "response_mime_type": "application/json",
    }

    model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
    system_instruction="Tu es un agent chargé d'analyser des tweets mentionnant la compte de Engie.\nIl te sera fournit le commentaire d'un utilisateur.\nTon rôle sera d'identifier et de renvoyer 9 facteurs :\n- \"Sentiment\" : \"Positif\" , \"Neutre\" ou \"Négatif\". Ce facteur doit identifier le sentiment du commentaire et renvoyer l'une des 3 possibilités (\"Positif\" , \"Neutre\" ou \"Négatif\"). L'utilisateur est positif quand il est engoué par un projet Engie, quand il est content d'un service ou d'un changement. Toutes les plaintes, réclamations, mécontentement sont négatif.\n- \"Problematique\" : \"Problèmes de facturation\", \"Pannes et urgences\", \"Service client injoignable\", \"Problèmes avec l’application\", \"Délai d’intervention\" ou \"aucune\". Ce facteur doit identifier le type de problème que rencontre le client et renvoyer l'une des 5 possibilités. Voici à quoi correspondent ces 5 possibilités :\nProblèmes de facturation : erreurs de montant, prélèvements injustifiés.\nPannes et urgences : absence de gaz, d’électricité, problème d’eau chaude.\nService client injoignable : absence de réponse, relances infructueuses.\nProblèmes avec l’application : bugs, indisponibilité du service.\nDélai d’intervention : retards dans la gestion des dossiers ou des réparations.\naucune : l'utilisateur n'a pas de problème.\n- \"Score\" : Calculer un score d’inconfort entre 0 et 100%.\n- \"Urgence\" : Mesure l'urgence de la situation de 0 à 10.\n- \"Reponse_automatique\" : Génère une réponse automatique au tweet. La réponse engage Engie et devra donc être respectueuse. En cas de problème, cette réponse doit s'excuser et proposer une solution si cela est possible (comme contacter le support Engie au 09 74 73 54 01). Remercier l'utilisateur s'il est content du service. Tu peux t'adapter à la situation. \n- \"Lieu\" : Si le lieu est mentionné, remplir cet emplacement avec le lieu concerné par le tweet, sinon laisser nul.\n- \"Reparabilite\" : Une valeur de 0 (pas de panne) à 5 (panne très couteuse ou complexe à résoudre).\n- \"Solution\" : Proposer une solution éventuelle en une ligne que l'entreprise pourrait réaliser pour résoudre le problème.\n- \"Type\" : Identifier le type de tweet :\n\"Positif\" : l'utilisateur est content du service que propose Engie, des ces engagements ou de ce que Engie organise.\n\"Plainte\" : l'utilisateur se plaint d'un problème.\n\"Question\" : l'utilisateur pose une question\n\nVoici le message utilisateur : ",
    )

    chat_session = model.start_chat(
    history=[
    ]
    )

    response = chat_session.send_message(tweet_text)

    analysis = json.loads(response.text)
    return analysis


import time

for index, row in cdf.iterrows():
    time.sleep(3)

    tweet_id = row['id']
    full_text = row['full_text']

    analysis = analyse_tweet(full_text)
    
    print(tweet_id, "  ", analysis)
    cdf.loc[index, "Sentiment"] = analysis["Sentiment"]
    cdf.loc[index, "Problematique"] = analysis["Problematique"]
    cdf.loc[index, "Score"] = analysis["Score"]
    cdf.loc[index, "Urgence"] = analysis["Urgence"]
    cdf.loc[index, "Reponse_automatique"] = analysis["Reponse_automatique"]
    cdf.loc[index, "Lieu"] = analysis["Lieu"]
    cdf.loc[index, "Reparabilite"] = analysis["Reparabilite"]
    cdf.loc[index, "Solution"] = analysis["Solution"]
    cdf.loc[index, "Type"] = analysis["Type"]

    # Sauvegarder le DataFrame nettoyé 
    cdf.to_csv('filtered_tweets_engie_cleaned.csv', index=False, sep=';')

1    {'Problematique': 'Délai d’intervention', 'Reparabilite': '2', 'Reponse_automatique': 'Bonjour, nous sommes sincèrement désolés pour le retard important concernant votre demande de devis. Nous comprenons votre frustration suite à l’absence de réponse de notre agence Engie Home Service de Grand Marseille à Aubagne malgré vos relances. Afin de résoudre cela rapidement, pourriez-vous nous contacter par message privé avec votre numéro de client ou votre adresse mail\xa0? Nous nous engageons à vous donner une réponse dans les plus brefs délais.', 'Score': 80, 'Sentiment': 'Négatif', 'Solution': 'Accélérer le processus de traitement des demandes de devis et améliorer la communication avec les clients.', 'Type': 'Plainte', 'Urgence': 7, 'Lieu': 'Aubagne'}


C:\Users\liena\AppData\Local\Temp\ipykernel_19016\1383953084.py:94: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  cdf.loc[index, "Reparabilite"] = analysis["Reparabilite"]


2    {'Problematique': 'Problèmes avec l’application', 'Reparabilite': '2', 'Reponse_automatique': "Bonjour ! Nous sommes désolés d'apprendre que vous rencontrez des difficultés avec l'application Mon Pilotage Elec. Nos équipes techniques travaillent actuellement à résoudre le problème.  En attendant, vous pouvez essayer de redémarrer votre téléphone ou tablette. Si le problème persiste, n'hésitez pas à contacter notre service client au 09 74 73 54 01.", 'Score': 70, 'Sentiment': 'Négatif', 'Solution': "Résoudre les bugs de l'application Mon Pilotage Elec.", 'Type': 'Plainte', 'Urgence': 7, 'Lieu': 'null'}
3    {'Problematique': 'Pannes et urgences', 'Reparabilite': '3', 'Reponse_automatique': 'Bonjour, nous sommes sincèrement désolés pour la situation que vous rencontrez concernant l’absence d’eau chaude dans votre copropriété. Nous comprenons que cela soit particulièrement difficile en plein hiver. Nous allons immédiatement contacter notre agence d’Amiens afin d’accélérer l’intervent

KeyboardInterrupt: 